In [1]:
# 2024-09-08
# Wonseok Hwang
# CC-BY-NC 4.0 International License

In [2]:
!pip install --quiet datasets
!pip install --quiet transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
!nvidia-smi

Wed Sep 11 05:23:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 1. Dataset

In [4]:
import datasets

dataset_card = "lbox/lbox_open"
task = "casename_classification"
data = datasets.load_dataset(dataset_card, task)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test2 split:   0%|          | 0/1294 [00:00<?, ? examples/s]

In [5]:
data.keys()

dict_keys(['train', 'validation', 'test', 'test2'])

In [6]:
data_v = data["validation"]
data_v[8]

{'id': 88,
 'casetype': 'criminal',
 'casename': '감염병의예방및관리에관한법률위반',
 'facts': "피고인은 경기 오산시 B, 2층에 있는 ‘C' 유흥주점을 운영하는 사람이다.\n경기도지사는 2020. 5. 10. ‘2020. 5. 10. 18:00경부터 2020. 5. 24. 24:00경까지 경기지역 내 다중시설(클럽형태, 유흥주점, 감성주점, 콜라텍)에 대해 집합을 금지한다'는 내용의 집합금지 행정명령을 내려 경기도청 담당 공무원은 피고인에게 집합금지 행정명령 알림문자를 발송하고, 오산시청 담당 공무원은 집합금지명령서를 피고인에게 교부하면서 이를 위 유흥주점 출입문에 부착하였다.\n그럼에도 불구하고 피고인은 2020. 5. 16. 21:20경 위 유흥주점에 찾아온 손님 D, E을 위 유흥주점에 입장시켰다.\n이로써 피고인은 경기도지사의 감염병 예방을 위한 집합금지명령을 위반하였다."}

In [7]:
len(data["train"]), len(data["validation"]), len(data["test"])

(8000, 1000, 1000)

# 1.1 Input

In [8]:
from transformers import AutoModel, AutoTokenizer
# https://huggingface.co/distilbert/distilbert-base-multilingual-cased에서 불러옴
backbone_card = "distilbert-base-multilingual-cased" # pretrained bert모델를 활용
backbone = AutoModel.from_pretrained(backbone_card)  # bert model로딩
tokenizer = AutoTokenizer.from_pretrained(backbone_card)

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
[x[0] for x in list(backbone.named_parameters())]

['embeddings.word_embeddings.weight',
 'embeddings.position_embeddings.weight',
 'embeddings.LayerNorm.weight',
 'embeddings.LayerNorm.bias',
 'transformer.layer.0.attention.q_lin.weight',
 'transformer.layer.0.attention.q_lin.bias',
 'transformer.layer.0.attention.k_lin.weight',
 'transformer.layer.0.attention.k_lin.bias',
 'transformer.layer.0.attention.v_lin.weight',
 'transformer.layer.0.attention.v_lin.bias',
 'transformer.layer.0.attention.out_lin.weight',
 'transformer.layer.0.attention.out_lin.bias',
 'transformer.layer.0.sa_layer_norm.weight',
 'transformer.layer.0.sa_layer_norm.bias',
 'transformer.layer.0.ffn.lin1.weight',
 'transformer.layer.0.ffn.lin1.bias',
 'transformer.layer.0.ffn.lin2.weight',
 'transformer.layer.0.ffn.lin2.bias',
 'transformer.layer.0.output_layer_norm.weight',
 'transformer.layer.0.output_layer_norm.bias',
 'transformer.layer.1.attention.q_lin.weight',
 'transformer.layer.1.attention.q_lin.bias',
 'transformer.layer.1.attention.k_lin.weight',
 'trans

In [11]:
# word embedding : 119547개의 token를 갖고, 각 token별로 768개의 차원을 갖는다.
backbone.embeddings.word_embeddings.weight.shape

torch.Size([119547, 768])

In [12]:
# in MB : # in MB : Word Embedding만 350MB를 차지함
119547 * 768 * 4 / 1024 / 1024

350.2353515625

In [13]:
backbone.embeddings.word_embeddings.weight.numel()

91812096

In [14]:
backbone.embeddings.word_embeddings.weight[0][0].dtype

torch.float32

In [15]:
facts = data_v[0]["facts"]
facts

"질병관리청장, 시·도지사 또는 시장·군수·구청장은 제1급 감염병이 발생한 경우 감염병의 전파방지 및 예방을 위하여 감염병의심자를 적당한 장소에 일정한 기간 격리시키는 조치를 하여야 하고, 그 격리조치를 받은 사람은 이를 위반하여서는 아니 된다.\n피고인은 해외에서 국내로 입국하였음을 이유로 2021. 4. 21.경 감염병의심자로 분류되었고, 같은 날 창녕군수로부터 ‘2021. 4. 21.부터 2021. 5. 5. 12:00경까지 피고인의 주거지인 경남 창녕군 B에서 격리해야 한다'는 내용의 자가격리 통지서를 수령하였다.\n1. 2021. 4. 27.자 범행\n그럼에도 불구하고 피고인은 2021. 4. 27. 11:20경에서 같은 날 11:59경까지 사이에 위 격리장소를 무단으로 이탈하여 자신의 승용차를 이용하여 경남 창녕군 C에 있는 ‘D' 식당에 다녀오는 등 자가격리 조치를 위반하였다.\n2. 2021. 5. 3.자 범행\n피고인은 2021. 5. 3. 10:00경에서 같은 날 11:35경까지 사이에 위 격리장소를 무단으로 이탈하여 자신의 승용차를 이용하여 불상의 장소를 다녀오는 등 자가격리 조치를 위반하였다."

In [16]:
# 문장을 토큰단위로 묶음 : 단어/문장단위는 선택
toks = tokenizer.tokenize(facts)[0:20]
print(toks)

['질', '##병', '##관', '##리', '##청', '##장', ',', '시', '·', '도', '##지', '##사', '또는', '시', '##장', '·', '군', '##수', '·', '구']


In [18]:
# 토큰의 임베딩id를 할당
ids = tokenizer.convert_tokens_to_ids(toks)
print(ids)

[9709, 73380, 20595, 12692, 40311, 13890, 117, 9485, 217, 9087, 12508, 12945, 20625, 9485, 13890, 217, 8910, 15891, 217, 8908]


In [19]:
import torch

# 각 토큰의 임베딩id에 매핑되는 임베딩값 추출
ids_tensor = torch.tensor(ids)
vecs = backbone.embeddings.word_embeddings(ids_tensor)
print(vecs)

tensor([[-0.0355, -0.0282,  0.0091,  ..., -0.0483,  0.0443, -0.0613],
        [-0.0229, -0.0365, -0.0619,  ..., -0.1163, -0.0326,  0.0328],
        [-0.0196, -0.0161, -0.1193,  ..., -0.0774,  0.0783, -0.0614],
        ...,
        [ 0.0671, -0.0503, -0.0432,  ...,  0.0116, -0.1382,  0.0555],
        [-0.0300, -0.0656, -0.0997,  ...,  0.0226, -0.0120,  0.0318],
        [-0.0051, -0.0861,  0.0107,  ...,  0.0059, -0.0111, -0.0063]],
       grad_fn=<EmbeddingBackward0>)


In [20]:
# 20개의 토큰에 해당하는 임베딩값(768차원)
vecs.shape

torch.Size([20, 768])

# 1.2 Output

In [24]:
from pprint import pprint

data_t = data["train"]
casenames = tuple(sorted(set(data_t["casename"]))) # casename확인용
pprint(casenames)

('감염병의예방및관리에관한법률위반',
 '강제추행',
 '건물명도(인도)',
 '건물인도',
 '게임산업진흥에관한법률위반',
 '공무집행방해',
 '공무집행방해, 상해',
 '공무집행방해, 업무방해',
 '공사대금',
 '공연음란',
 '공유물분할',
 '공직선거법위반',
 '교통사고처리특례법위반(치사)',
 '교통사고처리특례법위반(치상)',
 '교통사고처리특례법위반(치상), 도로교통법위반(음주운전)',
 '구상금',
 '근로기준법위반',
 '근로기준법위반, 근로자퇴직급여보장법위반',
 '근저당권말소',
 '기타(금전)',
 '대여금',
 '도로교통법위반(무면허운전)',
 '도로교통법위반(무면허운전), 도로교통법위반(음주운전)',
 '도로교통법위반(무면허운전), 도로교통법위반(음주운전), 자동차손해배상보장법위반',
 '도로교통법위반(무면허운전), 도로교통법위반(음주측정거부)',
 '도로교통법위반(사고후미조치), 도로교통법위반(음주운전), 특정범죄가중처벌등에관한법률위반(도주치상)',
 '도로교통법위반(사고후미조치), 특정범죄가중처벌등에관한법률위반(도주치상)',
 '도로교통법위반(음주운전)',
 '도로교통법위반(음주운전), 특정범죄가중처벌등에관한법률위반(위험운전치상)',
 '도로교통법위반(음주측정거부)',
 '마약류관리에관한법률위반(대마)',
 '마약류관리에관한법률위반(대마), 마약류관리에관한법률위반(향정)',
 '마약류관리에관한법률위반(향정)',
 '매매대금',
 '매매대금반환',
 '명예훼손',
 '모욕',
 '무고',
 '물품대금',
 '배당이의',
 '배상명령신청, 사기',
 '배상명령신청, 사기, 사문서위조, 위조사문서행사',
 '배상명령신청, 사기방조',
 '병역법위반',
 '보증금반환',
 '보험금',
 '보험사기방지특별법위반',
 '부당이득금',
 '사기',
 '사기, 사문서위조, 위조사문서행사',
 '사기방조',
 '사문서위조, 위조사문서행사',
 '사해행위취소',
 '산업안전보건법위반, 업무상과실치사',
 '상해',
 '상해, 폭행',
 '성매매

In [26]:
# 총 casename이 100개
len(casenames)

100

In [27]:
vecs.shape

torch.Size([20, 768])

In [28]:
# 총 casename이 100개이므로 pooking하겠다 -> 임베딩차원(768)을 100차원으로 임베딩하겠다
casename_pooler = torch.nn.Linear(768, 100)
casename_pooler.weight.shape

torch.Size([100, 768])

In [29]:
vecs[0].shape

torch.Size([768])

In [30]:
# 첫번째 토큰의 벡터값만 추출
logit = casename_pooler(vecs[0])
logit

tensor([-0.0859, -0.0052,  0.0181,  0.0190,  0.0011, -0.0236,  0.0131, -0.0048,
         0.0385, -0.0270, -0.0330, -0.0666,  0.0026, -0.0128,  0.0450,  0.0565,
         0.0050, -0.0070, -0.0681,  0.0211,  0.0371, -0.0095,  0.0190,  0.0370,
        -0.0383, -0.0029,  0.0142, -0.0306,  0.0042,  0.0701, -0.0093, -0.0265,
         0.0331,  0.0023, -0.0495, -0.0597, -0.0112, -0.0208,  0.0129, -0.0146,
        -0.0717,  0.0552, -0.0548,  0.0300,  0.0066, -0.0079, -0.0211, -0.0234,
         0.0047, -0.0691,  0.0086,  0.0219, -0.0233, -0.0060,  0.0038,  0.0271,
         0.0079, -0.0621,  0.0114, -0.0740,  0.0239, -0.0071,  0.0250,  0.0059,
        -0.0084, -0.0226, -0.0125, -0.0222, -0.0097,  0.0176, -0.0033, -0.0153,
         0.0467, -0.0157,  0.0321,  0.0129,  0.0078, -0.0295,  0.0581, -0.0472,
         0.0293, -0.0729,  0.0469,  0.0795,  0.0423, -0.0433, -0.0442,  0.0135,
        -0.0058,  0.0748, -0.0026,  0.0204, -0.0102, -0.0115,  0.0069, -0.0125,
        -0.0235, -0.0276,  0.0326,  0.03

In [31]:
# 768차원이 100차원으로 임베딩됨
logit.shape

torch.Size([100])

In [32]:
softmax = torch.nn.Softmax(dim=-1)
prob = softmax(logit)
print(prob)
print(torch.sum(prob))

tensor([0.0092, 0.0100, 0.0102, 0.0102, 0.0100, 0.0098, 0.0101, 0.0100, 0.0104,
        0.0098, 0.0097, 0.0094, 0.0100, 0.0099, 0.0105, 0.0106, 0.0101, 0.0099,
        0.0094, 0.0102, 0.0104, 0.0099, 0.0102, 0.0104, 0.0096, 0.0100, 0.0102,
        0.0097, 0.0101, 0.0107, 0.0099, 0.0098, 0.0104, 0.0100, 0.0095, 0.0094,
        0.0099, 0.0098, 0.0101, 0.0099, 0.0093, 0.0106, 0.0095, 0.0103, 0.0101,
        0.0099, 0.0098, 0.0098, 0.0101, 0.0093, 0.0101, 0.0102, 0.0098, 0.0100,
        0.0101, 0.0103, 0.0101, 0.0094, 0.0101, 0.0093, 0.0103, 0.0099, 0.0103,
        0.0101, 0.0099, 0.0098, 0.0099, 0.0098, 0.0099, 0.0102, 0.0100, 0.0099,
        0.0105, 0.0099, 0.0103, 0.0101, 0.0101, 0.0097, 0.0106, 0.0096, 0.0103,
        0.0093, 0.0105, 0.0108, 0.0104, 0.0096, 0.0096, 0.0102, 0.0100, 0.0108,
        0.0100, 0.0102, 0.0099, 0.0099, 0.0101, 0.0099, 0.0098, 0.0097, 0.0103,
        0.0104], grad_fn=<SoftmaxBackward0>)
tensor(1., grad_fn=<SumBackward0>)


In [33]:
-torch.log(prob[0])

tensor(4.6894, grad_fn=<NegBackward0>)

In [34]:
print(-torch.log(torch.tensor(0.01)))
print(-torch.log(torch.tensor(0.99)))

tensor(4.6052)
tensor(0.0101)


In [36]:
loss_fn = torch.nn.CrossEntropyLoss()
label = torch.Tensor([1] + [0]*99)
loss_fn(logit, label)

tensor(4.6894, grad_fn=<DivBackward1>)

In [40]:
# 임의의 레이블 설정
label

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [41]:
# logit=2이면 loss가 2.6
logit[0] = 2
loss_fn(logit, label)

tensor(2.6663, grad_fn=<DivBackward1>)

In [42]:
logit[0] = 20
loss_fn(logit, label)

tensor(1.1921e-07, grad_fn=<DivBackward1>)

# 1.3 Entropy

# 2. Training

In [43]:
import torch

# 무작위성을 제어해서 variance를 줄임
torch.manual_seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps") # mac전용 GPU

# 2.1. Data
dataloader_t = torch.utils.data.DataLoader(
    data_t,
    batch_size=36,
    shuffle=True,
)

dataloader_v = torch.utils.data.DataLoader(
    data_v,
    batch_size=36,
    shuffle=False,
)

In [44]:
device

device(type='cuda')

In [45]:
for batch in dataloader_t:
    print(batch)
    break

{'id': tensor([9026,   72, 9825, 1632, 5605, 1621,  951, 4714, 2048, 1542, 5341, 8056,
        2206, 3333, 3509, 6518, 7251, 1602, 1559, 4168, 7668,  375, 3011, 7921,
        6527, 9749, 6276, 6403, 6103, 6452, 6146, 1964,  423, 5445, 8860, 4927]), 'casetype': ['civil', 'criminal', 'criminal', 'criminal', 'criminal', 'criminal', 'criminal', 'civil', 'civil', 'civil', 'criminal', 'criminal', 'criminal', 'civil', 'criminal', 'civil', 'criminal', 'criminal', 'civil', 'criminal', 'civil', 'civil', 'criminal', 'criminal', 'civil', 'criminal', 'civil', 'civil', 'civil', 'civil', 'civil', 'civil', 'criminal', 'criminal', 'civil', 'criminal'], 'casename': ['토지인도', '감염병의예방및관리에관한법률위반', '협박', '근로기준법위반', '성매매알선등행위의처벌에관한법률위반(성매매알선등)', '근로기준법위반', '공연음란', '부당이득금', '대여금', '구상금', '산업안전보건법위반, 업무상과실치사', '전자금융거래법위반', '도로교통법위반(무면허운전), 도로교통법위반(음주운전)', '매매대금', '명예훼손', '손해배상(의)', '업무방해, 폭행', '근로기준법위반', '구상금', '배상명령신청, 사기, 사문서위조, 위조사문서행사', '임금', '건물인도', '마약류관리에관한법률위반(대마)', '전기통신사업법위반', '손해배상(의)', '폭행', '손해배상',

In [46]:
# batch별 데이터 정보
batch.keys()

dict_keys(['id', 'casetype', 'casename', 'facts'])

In [47]:
class MyModel(torch.nn.Module):

    # backbone : pretrained bert
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.pooler = torch.nn.Linear(768, 100)

    def forward(self, input_ids, attention_mask):
        # attention_mask : batch내에서도 size가 다를 수도 있기 때문에 mask를 수행함
        encoded = self.backbone(input_ids, attention_mask)
        last = encoded.last_hidden_state # [batch, 768]
        logit = self.pooler(last[:, 0]) # [batch, 100] : 0번째 token만 중요한 정보이기 때문에 0번째 토큰의 logit값만 추출하겠다.
        return logit

In [49]:
model = MyModel(backbone)
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

In [50]:
from tqdm import tqdm

model = model.train()
for epoch in (range(2)):
    tot_loss = 0
    for i, batch in enumerate(tqdm(dataloader_t)):
        xs = batch["facts"] # input : 자연어(추후에 벡터로 변형)
        ys = batch["casename"] # label : 사건유형
        ys = [casenames.index(y) for y in ys]

        # input_features : inputs_ids, attention_mask
        # input_ids : 각 자연어를 토큰id로 변형(mask token은 token_id 0으로 지정)
        # attenton_mask : masking할 개수
        # truncation : 입력 시퀀스가 모델의 길이를 초과하면 초과부분을 잘라냄
        # return_tensors : pt는 pytorch tensor형태로 반환
        input_features = tokenizer(xs, padding=True, truncation=True, return_tensors="pt")
        input_ids = input_features["input_ids"].to(device)
        attention_mask = input_features["attention_mask"].to(device)
        label = torch.tensor(ys).to(device)

        logit = model.forward(input_ids, attention_mask)

        optimizer.zero_grad()

        loss1 = loss_fn(logit, label)
        tot_loss += loss1.item()
        loss1.backward()
        optimizer.step() # parameter update
        if i % 10 == 0:
            print(f"epoch: {epoch}, batch: {i}, loss: {tot_loss / (i+1)}")

    print(f"epoch: {epoch}, tot_loss: {tot_loss}")
        # attention_mask = encoded["attention_mask"].to(device)
    # break
    # input_ids = data1["input_ids"].to(device)
    # label = data1["label"].to(device)
    # logit = model(input_ids)
    # loss = loss_fn(logit, label)
    # print(loss)
    # break

  0%|          | 1/223 [00:03<11:59,  3.24s/it]

epoch: 0, batch: 0, loss: 4.5876994132995605


  5%|▍         | 11/223 [00:17<05:16,  1.50s/it]

epoch: 0, batch: 10, loss: 4.641515775160356


  9%|▉         | 21/223 [00:32<05:05,  1.51s/it]

...

epoch: 1, batch: 210, loss: 1.219341227511094


 99%|█████████▉| 221/223 [06:05<00:03,  1.66s/it]

epoch: 1, batch: 220, loss: 1.2049732631687664


100%|██████████| 223/223 [06:08<00:00,  1.65s/it]

epoch: 1, tot_loss: 268.1608998179436


In [51]:
ys

[56, 4, 13, 77, 28, 53, 90, 72]

In [ ]:
device = "cpu"
model = model.to(device)
cnt = 0
for batch in dataloader_v:
  cnt += 1
  model = model.eval()
  xs = batch["facts"]
  ys = batch["casename"]
  ys = [casenames.index(y) for y in ys]

  input_features = tokenizer(xs, padding=True, truncation=True, return_tensors="pt")
  input_ids = input_features["input_ids"].to(device)
  attention_mask = input_features["attention_mask"].to(device)
  label = torch.tensor(ys).to(device)

  logit = model.forward(input_ids, attention_mask)
  preds = logit.argmax(-1)
  print(f"GT: {label}")
  print(f"PR: {preds}")
  if cnt > 10:
    break

GT: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3])
PR: tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1, 95,
         1,  1,  3, 77, 77, 77, 77, 90, 77,  3, 10, 10,  3,  3,  3,  3, 60,  3])


In [ ]:
# Prediction without training

In [ ]:
device = "cpu"
model = model.to(device)
cnt = 0
for batch in dataloader_v:
  cnt += 1
  model = model.eval()
  xs = batch["facts"]
  ys = batch["casename"]
  ys = [casenames.index(y) for y in ys]

  input_features = tokenizer(xs, padding=True, truncation=True, return_tensors="pt")
  input_ids = input_features["input_ids"].to(device)
  attention_mask = input_features["attention_mask"].to(device)
  label = torch.tensor(ys).to(device)

  logit = model.forward(input_ids, attention_mask)
  preds = logit.argmax(-1)
  print(f"GT: {label}")
  print(f"PR: {preds}")
  if cnt > 10:
    break